In [ ]:
!pip -q install git+https://github.com/mwshinn/PyDDM
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from pyddm import Sample

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
subjects

f"s{subID}", f"s{subID}_results.txt"
os.path.join(f"s{subID}", f"s{subID}_results.txt")
os.path.join(f"s{subID}_results.txt")

,RT
0,False
1,False
2,False
3,False
4,False
...,...
245,False
246,False
247,True
248,False


In [24]:
# Read the tidy data file
df = pd.read_csv('inference_tidy.csv')
df = df.dropna(subset=['RT'])
subjects = df['subID'].unique()

# Create output directory
output_dir = 'plots/'
os.makedirs("plots", exist_ok=True)

for subID in subjects:
    # Get subject-specific timing parameters from dataframe
    subject_df = df[df['subID'] == subID]
    if subject_df.empty:
        print(f"No data found for subject {subID}, skipping.")
        continue

    # Extract timing parameters
    signal1_onset = subject_df['signal1_onset'].mode()[0]
    noise2_onset = subject_df['noise2_onset'].mode()[0]
    signal2_onset = subject_df['signal2_onset'].mode()[0]

    # Read results file
    results_file = os.path.join(f"s{subID}_results.txt")
    if not os.path.exists(results_file):
        print(f"Results file not found for subject {subID}, skipping.")
        continue

    with open(results_file, 'r') as f:
        content = f.read()

    # Extract loss
    loss_match = re.search(r'Loss: ([\d\.]+)', content)
    loss = float(loss_match.group(1)) if loss_match else None

    # Extract parameters
    params = {}
    for name, value in re.findall(r"'([\w_]+)': Fitted\(([\d\.]+)", content):
        params[name] = float(value)

    # specify drift function
    def eightDrifts(t, trueCongruence, signal1_onset, noise2_onset, signal2_onset,
                    noise1Drift_80, noise1Drift_50, signal1Drift_80, signal1Drift_50,
                    noise2Drift_80, noise2Drift_50, signal2Drift_80, signal2Drift_50):
        # First noise period
        if t < signal1_onset:
            if trueCongruence == 'congruent': return noise1Drift_80
            elif trueCongruence == 'incongruent': return -noise1Drift_80
            else: return noise1Drift_50
        # First signal period
        elif t < noise2_onset:
            if trueCongruence == 'congruent': return signal1Drift_80
            elif trueCongruence == 'incongruent': return -signal1Drift_80
            else: return signal1Drift_50
        # Second noise period
        elif t < signal2_onset:
            if trueCongruence == 'congruent': return noise2Drift_80
            elif trueCongruence == 'incongruent': return -noise2Drift_80
            else: return noise2Drift_50
        # Second signal period
        else:
            if trueCongruence == 'congruent': return signal2Drift_80
            elif trueCongruence == 'incongruent': return -signal2Drift_80
            else: return signal2Drift_50

        # initialize model object
    model = pyddm.gddm(
    drift=eightDrifts,
    name=f"eightDrifts_s{subID}",
    starting_position=0,
    bound='B',
    T_dur=4.1,
    nondecision='nondectime',
    parameters=params,
    conditions=['trueCongruence', 'coherence', 'signal1_onset', 'noise2_onset', 'signal2_onset'])

    # create pyDDM sample object
    sample = pyddm.Sample.from_pandas_dataframe(subject_df, rt_column_name='RT', choice_column_name='accuracy')

    pyddm.Sample.from_pandas_dataframe(df, rt_column_name='RT', choice_column_name='accuracy')

    # pyddm.plot.plot_fit_diagnostics(model, data, fig=fig)

    # Create figure
    fig = plt.figure(figsize=(12, 10))
    # Add noise regions to each subplot
    for ax in fig.get_axes():
        try:
            # First noise period (0 to signal1_onset)
            ax.axvspan(0, signal1_onset, alpha=0.25, color='lightblue')
            # Second noise period (noise2_onset to signal2_onset)
            ax.axvspan(noise2_onset, signal2_onset, alpha=0.25, color='lightblue')

            # Add labels
            ylim = ax.get_ylim()
            ax.text(signal1_onset/2, ylim[1]*0.95, 'Noise 1',
                    ha='center', va='top', backgroundcolor='white', alpha=0.7)
            ax.text((noise2_onset + signal2_onset)/2, ylim[1]*0.95, 'Noise 2',
                    ha='center', va='top', backgroundcolor='white', alpha=0.7)
        except Exception as e:
            print(f"Warning: Could not add noise regions to a subplot: {e}")

    # Create parameter text
    param_text = """
    Fitted Parameters for Subject {subID} (Loss: {loss:.4f})
    Boundary (B): {params.get('bound', 'N/A'):.3f}
    Non-decision time: {params.get('nondectime', 'N/A'):.3f}

    First Period Drifts:
    noise1Drift_80: {params.get('noise1Drift_80', 'N/A'):.3f}  noise1Drift_50: {params.get('noise1Drift_50', 'N/A'):.3f}
    signal1Drift_80: {params.get('signal1Drift_80', 'N/A'):.3f}  signal1Drift_50: {params.get('signal1Drift_50', 'N/A'):.3f}

    Second Period Drifts:
    noise2Drift_80: {params.get('noise2Drift_80', 'N/A'):.3f}  noise2Drift_50: {params.get('noise2Drift_50', 'N/A'):.3f}
    signal2Drift_80: {params.get('signal2Drift_80', 'N/A'):.3f}  signal2Drift_50: {params.get('signal2Drift_50', 'N/A'):.3f}

    Timing: signal1_onset={signal1_onset:.2f}, noise2_onset={noise2_onset:.2f}, signal2_onset={signal2_onset:.2f}
    """

    # Add text box and finalize plot
    fig.text(0.5, 0.01, param_text, ha='center', va='bottom',
              bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
              fontsize=10, fontfamily='monospace')
    plt.tight_layout(rect=[0, 0.15, 1, 0.95])
    plt.suptitle(f'Subject {subID} Model Fit', fontsize=16, y=0.98)

    pyddm.plot.plot_fit_diagnostics(model, sample, data_dt=0.05, fig=fig)
    # Save and close
    plt.savefig(os.path.join(output_dir, f'subject_{subID}_fit.png'), dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Successfully created plot for subject {subID}")

Successfully created plot for subject 32
Successfully created plot for subject 33
Successfully created plot for subject 35
Successfully created plot for subject 36
Successfully created plot for subject 37
Successfully created plot for subject 38
Successfully created plot for subject 39
Successfully created plot for subject 40
Successfully created plot for subject 41
Successfully created plot for subject 42
Successfully created plot for subject 43
Successfully created plot for subject 44
Successfully created plot for subject 45
Successfully created plot for subject 46
Successfully created plot for subject 47
Successfully created plot for subject 48
Successfully created plot for subject 49
Successfully created plot for subject 50
Successfully created plot for subject 51
Successfully created plot for subject 52
Successfully created plot for subject 53
Successfully created plot for subject 54


In [20]:
subID
subject_df
np.isnan(subject_df['RT'])

,RT
0,False
1,False
2,False
3,False
4,False
...,...
245,False
246,False
247,True
248,False
